In [2]:
pip install numpy==1.26.4 torch transformers==4.48.2 scikit-learn accelerate==0.26.0 matplotlib tqdm pandas seaborn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 971.9 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 507.1 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 315.8 kB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 288.8 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.7/270.7 kB 275.6 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 692.6 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.3/515.3 kB 433.8 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 kB 277.9 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 349.4 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import torch
print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

2.2.0+cu121
12.1
True
NVIDIA GeForce RTX 3090


In [5]:
import os, glob, gc, copy, math, random, time, json
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score
from tqdm import tqdm

# Configuration
DATA_DIR    = "/workspace/SPLIT_BEATS_NPY/train"
LABEL_MAP   = {'N':0,'L':1,'R':2,'V':3,'Q':4}
SEED        = 42
N_SPLITS    = 2
EPOCHS      = 20
BATCH_SIZE  = 32
MAX_LEN     = 512
EMB_DIM     = 512
N_HEADS     = 8
FF_DIM      = 2048
N_LAYERS    = 12
LR          = 2e-5
OUTPUT_BASE = "/workspace/HASIL_DECODER/HASIL_2"
os.makedirs(OUTPUT_BASE, exist_ok=True)

random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
PAD_ID, CLS_ID  = 256, 257
VOCAB_SIZE      = 258
cls_names       = list(LABEL_MAP.keys())

def signal_to_ids(sig: np.ndarray):
    norm = ((sig - sig.min()) / (sig.ptp() + 1e-8) * 255).astype(int)
    ids  = np.concatenate(([CLS_ID], norm))[:MAX_LEN]
    mask = np.ones_like(ids, dtype=int)
    if len(ids) < MAX_LEN:
        pad_len = MAX_LEN - len(ids)
        ids  = np.concatenate((ids,  np.full(pad_len, PAD_ID)))
        mask = np.concatenate((mask, np.zeros(pad_len)))
    return ids, mask

# Load data
files, labels = [], []
for cls, idx in LABEL_MAP.items():
    for f in glob.glob(os.path.join(DATA_DIR, cls, "*.npy")):
        files.append(f); labels.append(idx)
files, labels = np.array(files), np.array(labels)

all_ids, all_mask = [], []
for f in tqdm(files, desc="Pre-encoding"):
    ids, msk = signal_to_ids(np.load(f))
    all_ids.append(ids);  all_mask.append(msk)
all_ids  = torch.tensor(all_ids,  dtype=torch.long)
all_mask = torch.tensor(all_mask, dtype=torch.long)
labels_t = torch.tensor(labels,   dtype=torch.long)

class DecoderOnlyClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_emb = nn.Embedding(VOCAB_SIZE, EMB_DIM, padding_idx=PAD_ID)
        self.pos_emb   = nn.Parameter(torch.zeros(1, MAX_LEN, EMB_DIM))
        dec_layer = nn.TransformerDecoderLayer(d_model=EMB_DIM, nhead=N_HEADS,
                                               dim_feedforward=FF_DIM, dropout=0.1,
                                               batch_first=True)
        self.decoder = nn.TransformerDecoder(dec_layer, num_layers=N_LAYERS)
        self.fc      = nn.Linear(EMB_DIM, len(LABEL_MAP))
        nn.init.normal_(self.pos_emb, std=0.02)

    def forward(self, input_ids, attention_mask):
        tgt = self.token_emb(input_ids) + self.pos_emb
        memory = torch.zeros_like(tgt).to(tgt.device)
        x = self.decoder(tgt, memory, tgt_key_padding_mask=~attention_mask.bool())
        x = (x * attention_mask.unsqueeze(-1)).sum(1) / attention_mask.sum(1, keepdim=True).clamp(min=1e-9)
        return self.fc(x)

skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
rows_all = []

for fold, (tr, va) in enumerate(skf.split(all_ids, labels), 1):
    print(f"\n==== FOLD {fold}/{N_SPLITS} ====")
    gc.collect(); torch.cuda.empty_cache()

    train_ds = TensorDataset(all_ids[tr], all_mask[tr], labels_t[tr])
    val_ds   = TensorDataset(all_ids[va], all_mask[va], labels_t[va])
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE)

    model = DecoderOnlyClassifier().to(DEVICE)

    for param in model.parameters():
        param.requires_grad = False
    model.fc.weight.requires_grad = True
    model.fc.bias.requires_grad = True

    optim = torch.optim.AdamW(model.fc.parameters(), lr=LR)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optim, EPOCHS)

    best_state, best_loss = None, math.inf
    out_dir = os.path.join(OUTPUT_BASE, f"fold{fold}"); os.makedirs(out_dir, exist_ok=True)

    for epoch in range(1, EPOCHS + 1):
        print(f"\n⏰ Fold {fold} | Epoch {epoch}/{EPOCHS}")
        model.train(); total_loss = 0.0; correct = 0; total = 0
        for ids, msk, lbl in tqdm(train_loader, leave=False):
            ids, msk, lbl = ids.to(DEVICE), msk.to(DEVICE), lbl.to(DEVICE)
            optim.zero_grad(); out = model(ids, msk)
            loss = F.cross_entropy(out, lbl)
            loss.backward(); torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optim.step(); total_loss += loss.item()
            correct += (out.argmax(1) == lbl).sum().item()
            total += lbl.size(0)
        train_acc = correct / total
        scheduler.step()

        model.eval(); val_loss, correct_val, total_val = 0.0, 0, 0
        with torch.no_grad():
            for ids, msk, lbl in val_loader:
                ids, msk, lbl = ids.to(DEVICE), msk.to(DEVICE), lbl.to(DEVICE)
                out = model(ids, msk)
                val_loss += F.cross_entropy(out, lbl, reduction='sum').item()
                correct_val += (out.argmax(1) == lbl).sum().item()
                total_val += lbl.size(0)
        val_loss /= len(val_ds)
        val_acc = correct_val / total_val

        print(f"Train Loss: {total_loss/len(train_loader):.4f}, Train Acc: {train_acc:.4f}")
        print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

        if val_loss < best_loss:
            best_loss, best_state = val_loss, copy.deepcopy(model.state_dict())

    torch.save(best_state, os.path.join(out_dir, "best_model.pt"))
    print(f"✔️ Fold {fold} Training Completed.")

print("\n✔️ All training folds completed.")

Pre-encoding: 100%|██████████| 28000/28000 [00:03<00:00, 9139.91it/s]



==== FOLD 1/2 ====

⏰ Fold 1 | Epoch 1/20


Train Loss: 1.6076, Train Acc: 0.2178
Val Loss: 1.5765, Val Acc: 0.2924

⏰ Fold 1 | Epoch 2/20


Train Loss: 1.5544, Train Acc: 0.3308
Val Loss: 1.5302, Val Acc: 0.3850

⏰ Fold 1 | Epoch 3/20


Train Loss: 1.5153, Train Acc: 0.3888
Val Loss: 1.4956, Val Acc: 0.4054

⏰ Fold 1 | Epoch 4/20


...

Train Loss: 1.3350, Train Acc: 0.4696
Val Loss: 1.3363, Val Acc: 0.4679

⏰ Fold 2 | Epoch 17/20


Train Loss: 1.3334, Train Acc: 0.4660
Val Loss: 1.3356, Val Acc: 0.4683

⏰ Fold 2 | Epoch 18/20


Train Loss: 1.3330, Train Acc: 0.4688
Val Loss: 1.3351, Val Acc: 0.4683

⏰ Fold 2 | Epoch 19/20


Train Loss: 1.3323, Train Acc: 0.4701
Val Loss: 1.3349, Val Acc: 0.4679

⏰ Fold 2 | Epoch 20/20


Train Loss: 1.3325, Train Acc: 0.4674
Val Loss: 1.3348, Val Acc: 0.4680
✔️ Fold 2 Training Completed.

✔️ All training folds completed.


In [ ]:
import os, glob, gc, copy, math, random, time, json
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score
from tqdm import tqdm

# Configuration
DATA_DIR    = "/workspace/SPLIT_BEATS_NPY/train"
LABEL_MAP   = {'N':0,'L':1,'R':2,'V':3,'Q':4}
SEED        = 42
N_SPLITS    = 2
EPOCHS      = 20
BATCH_SIZE  = 32
MAX_LEN     = 512
EMB_DIM     = 512
N_HEADS     = 8
FF_DIM      = 2048
N_LAYERS    = 12
LR          = 2e-5
OUTPUT_BASE = "/workspace/HASIL_DECODER/HASIL_TUNING_FULL_2"
os.makedirs(OUTPUT_BASE, exist_ok=True)

random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
PAD_ID, CLS_ID  = 256, 257
VOCAB_SIZE      = 258
cls_names       = list(LABEL_MAP.keys())

def signal_to_ids(sig: np.ndarray):
    norm = ((sig - sig.min()) / (sig.ptp() + 1e-8) * 255).astype(int)
    ids  = np.concatenate(([CLS_ID], norm))[:MAX_LEN]
    mask = np.ones_like(ids, dtype=int)
    if len(ids) < MAX_LEN:
        pad_len = MAX_LEN - len(ids)
        ids  = np.concatenate((ids,  np.full(pad_len, PAD_ID)))
        mask = np.concatenate((mask, np.zeros(pad_len)))
    return ids, mask

# Load data
files, labels = [], []
for cls, idx in LABEL_MAP.items():
    for f in glob.glob(os.path.join(DATA_DIR, cls, "*.npy")):
        files.append(f); labels.append(idx)
files, labels = np.array(files), np.array(labels)

all_ids, all_mask = [], []
for f in tqdm(files, desc="Pre-encoding"):
    ids, msk = signal_to_ids(np.load(f))
    all_ids.append(ids);  all_mask.append(msk)
all_ids  = torch.tensor(all_ids,  dtype=torch.long)
all_mask = torch.tensor(all_mask, dtype=torch.long)
labels_t = torch.tensor(labels,   dtype=torch.long)

class DecoderOnlyClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_emb = nn.Embedding(VOCAB_SIZE, EMB_DIM, padding_idx=PAD_ID)
        self.pos_emb   = nn.Parameter(torch.zeros(1, MAX_LEN, EMB_DIM))
        dec_layer = nn.TransformerDecoderLayer(d_model=EMB_DIM, nhead=N_HEADS,
                                               dim_feedforward=FF_DIM, dropout=0.1,
                                               batch_first=True)
        self.decoder = nn.TransformerDecoder(dec_layer, num_layers=N_LAYERS)
        self.fc      = nn.Linear(EMB_DIM, len(LABEL_MAP))
        nn.init.normal_(self.pos_emb, std=0.02)

    def forward(self, input_ids, attention_mask):
        tgt = self.token_emb(input_ids) + self.pos_emb
        memory = torch.zeros_like(tgt).to(tgt.device)
        x = self.decoder(tgt, memory, tgt_key_padding_mask=~attention_mask.bool())
        x = (x * attention_mask.unsqueeze(-1)).sum(1) / attention_mask.sum(1, keepdim=True).clamp(min=1e-9)
        return self.fc(x)

skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
rows_all = []

for fold, (tr, va) in enumerate(skf.split(all_ids, labels), 1):
    print(f"\n==== FOLD {fold}/{N_SPLITS} ====")
    gc.collect(); torch.cuda.empty_cache()

    train_ds = TensorDataset(all_ids[tr], all_mask[tr], labels_t[tr])
    val_ds   = TensorDataset(all_ids[va], all_mask[va], labels_t[va])
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE)

    model = DecoderOnlyClassifier().to(DEVICE)

    # FULL TUNING: semua parameter di-train
    for param in model.parameters():
        param.requires_grad = True

    optim = torch.optim.AdamW(model.parameters(), lr=LR)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optim, EPOCHS)

    best_state, best_loss = None, math.inf
    out_dir = os.path.join(OUTPUT_BASE, f"fold{fold}"); os.makedirs(out_dir, exist_ok=True)

    history = []

    for epoch in range(1, EPOCHS + 1):
        model.train(); total_loss = 0.0; correct = 0; total = 0
        for ids, msk, lbl in tqdm(train_loader, leave=False):
            ids, msk, lbl = ids.to(DEVICE), msk.to(DEVICE), lbl.to(DEVICE)
            optim.zero_grad(); out = model(ids, msk)
            loss = F.cross_entropy(out, lbl)
            loss.backward(); torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optim.step(); total_loss += loss.item()
            correct += (out.argmax(1) == lbl).sum().item()
            total += lbl.size(0)
        train_acc = correct / total
        scheduler.step()

        model.eval(); val_loss, correct_val, total_val = 0.0, 0, 0
        with torch.no_grad():
            for ids, msk, lbl in val_loader:
                ids, msk, lbl = ids.to(DEVICE), msk.to(DEVICE), lbl.to(DEVICE)
                out = model(ids, msk)
                val_loss += F.cross_entropy(out, lbl, reduction='sum').item()
                correct_val += (out.argmax(1) == lbl).sum().item()
                total_val += lbl.size(0)
        val_loss /= len(val_ds)
        val_acc = correct_val / total_val

        print(f"Epoch {epoch}/{EPOCHS} | Train Loss: {total_loss/len(train_loader):.4f} | Train Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

        history.append({
            'epoch': epoch,
            'train_loss': total_loss/len(train_loader),
            'train_acc': train_acc,
            'val_loss': val_loss,
            'val_acc': val_acc
        })

        if val_loss < best_loss:
            best_loss, best_state = val_loss, copy.deepcopy(model.state_dict())

    # Simpan model & history
    torch.save(best_state, os.path.join(out_dir, "best_model.pt"))
    pd.DataFrame(history).to_csv(os.path.join(out_dir, "history.csv"), index=False)
    print(f"✔️ Fold {fold} Training Completed. Best Val Loss: {best_loss:.4f}")

print("\n✔️ All training folds completed.")


Pre-encoding: 100%|██████████| 28000/28000 [00:03<00:00, 8625.28it/s]
/tmp/ipykernel_3606/511644460.py:52: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  all_ids  = torch.tensor(all_ids,  dtype=torch.long)



==== FOLD 1/2 ====


Epoch 1/20 | Train Loss: 0.9732 | Train Acc: 0.6345 | Val Loss: 0.7783 | Val Acc: 0.7276


Epoch 2/20 | Train Loss: 0.6535 | Train Acc: 0.7701 | Val Loss: 0.6036 | Val Acc: 0.7868


Epoch 3/20 | Train Loss: 0.5097 | Train Acc: 0.8248 | Val Loss: 0.5122 | Val Acc: 0.8212


...

Epoch 5/20 | Train Loss: 0.3789 | Train Acc: 0.8714 | Val Loss: 0.4161 | Val Acc: 0.8605


Epoch 6/20 | Train Loss: 0.3309 | Train Acc: 0.8884 | Val Loss: 0.4066 | Val Acc: 0.8702


Epoch 7/20 | Train Loss: 0.2993 | Train Acc: 0.8981 | Val Loss: 0.3939 | Val Acc: 0.8679


Epoch 8/20 | Train Loss: 0.2572 | Train Acc: 0.9139 | Val Loss: 0.3108 | Val Acc: 0.9007


 11%|█         | 46/438 [00:27<03:52,  1.69it/s]